In [1]:
import torch


def get_paralleldb(db,remove_index):

    return torch.cat((db[0:remove_index],db[remove_index+1:]))

def get_paralleldbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_paralleldb(db,i)
        parallel_dbs.append(pdb)

    return parallel_dbs
def create_db_and_parallels(num_entries):

    db = torch.rand(num_entries)>0.5
    pdbs=get_paralleldbs(db)

    return db,pdbs



def sensitivity(query,n_entries=1000):
    db, pdbs = create_db_and_parallels(n_entries)

    full_db_result = query(db)

    max_distance=0
    for pdb in pdbs:
        pdb_result=query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)
        if(db_distance > max_distance):
            max_distance=db_distance

    return max_distance


def query(db,threshold=5):
    return (db.sum()>threshold).float()

for i in range(10):
    sens_f=sensitivity(query,n_entries=10)
    print(sens_f)








0
0
tensor(1.)
0
0
0
0
0
0
0


In [27]:
def query(db):
    
    true_result = torch.mean(db.float())

    first_coin_flip = (torch.rand(len(db))<0.5).float() 
    second_coin_flip = (torch.rand(len(db))<0.5).float()
    
    augmented_db = db.float()*first_coin_flip+ (1-first_coin_flip)*second_coin_flip
    
    db_result = torch.mean(augmented_db.float())*2-0.5
    
    return db_result,true_result

    

In [29]:
db , pdbs = create_db_and_parallels(10)
private_result,true_result = query(db)
print("With Noise:"+ str(private_result))
print("Without Noise:"+ str(true_result))

With Noise:tensor(0.7000)
Without Noise:tensor(0.4000)


In [30]:
db , pdbs = create_db_and_parallels(100)
private_result,true_result = query(db)
print("With Noise:"+ str(private_result))
print("Without Noise:"+ str(true_result))

With Noise:tensor(0.3600)
Without Noise:tensor(0.4200)


In [33]:
db , pdbs = create_db_and_parallels(1000)
private_result,true_result = query(db)
print("With Noise:"+ str(private_result))
print("Without Noise:"+ str(true_result))

With Noise:tensor(0.4720)
Without Noise:tensor(0.4820)


In [32]:
db , pdbs = create_db_and_parallels(1000)
private_result,true_result = query(db)
print("With Noise:"+ str(private_result))
print("Without Noise:"+ str(true_result))

With Noise:tensor(0.4720)
Without Noise:tensor(0.4870)


In [34]:
def query(db, noise=0.2):
    
    true_result = torch.mean(db.float())

    first_coin_flip = (torch.rand(len(db)) < noise).float()
    second_coin_flip = (torch.rand(len(db)) < 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    sk_result = augmented_database.float().mean()

    private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)

    return private_result, true_result

In [35]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.1)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5111)
Without Noise:tensor(0.5100)


In [36]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.2)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5000)
Without Noise:tensor(0.5900)


In [38]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.4)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5000)
Without Noise:tensor(0.4900)


In [41]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.8)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.6000)
Without Noise:tensor(0.5600)


In [43]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db, noise=0.8)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5110)
Without Noise:tensor(0.5049)


In [58]:
epsilon = 0.5

In [59]:
import numpy as np

In [60]:
db, pdbs = create_db_and_parallels(100)

In [61]:
def sum_query(db):
    return db.sum()


In [66]:
def mean_query(db):
    return torch.mean(db.float())

In [67]:
def laplacian_mechanism(db,query,sensitivity):
    
    beta = sensitivity/epsilon
    
    noise = torch.tensor(np.random.laplace(0,beta,1))
    
    return query(db)+noise

In [77]:
laplacian_mechanism(db,sum_query,1)

tensor([43.6560], dtype=torch.float64)

In [78]:
laplacian_mechanism(db,mean_query,1/100)

tensor([0.4790], dtype=torch.float64)

In [79]:
db.sum()

tensor(43)

In [80]:
torch.mean(db.float())


tensor(0.4300)